In [ ]:
import sys
import os
import onnx
import tensorflow as tf
import numpy as np
import tf2onnx

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2024-11-21 18:32:03.975200: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
data = load_iris()
X = data.data  
y = data.target

In [4]:
X = X[y < 2]  # Select only classes 0 and 1
y = y[y < 2]

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Neural network
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(10, activation='relu', input_shape=(4,)),  # 4 features for input
    tf.keras.layers.Dense(5, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])


2024-11-21 18:32:57.466576: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [6]:
# Compile and train model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=4, validation_split=0.2)

Epoch 1/5
16/16 [==============================] - 1s 30ms/step - loss: 0.6408 - accuracy: 0.8750 - val_loss: 0.6167 - val_accuracy: 1.0000
Epoch 2/5
16/16 [==============================] - 0s 12ms/step - loss: 0.6147 - accuracy: 0.9844 - val_loss: 0.5915 - val_accuracy: 1.0000
Epoch 3/5
16/16 [==============================] - 0s 12ms/step - loss: 0.5915 - accuracy: 1.0000 - val_loss: 0.5663 - val_accuracy: 1.0000
Epoch 4/5
16/16 [==============================] - 0s 14ms/step - loss: 0.5685 - accuracy: 1.0000 - val_loss: 0.5402 - val_accuracy: 1.0000
Epoch 5/5
16/16 [==============================] - 0s 9ms/step - loss: 0.5438 - accuracy: 1.0000 - val_loss: 0.5132 - val_accuracy: 1.0000


In [7]:
tf.saved_model.save(model, "iris_binary_classification_model_tf")

INFO:tensorflow:Assets written to: iris_binary_classification_model_tf/assets


INFO:tensorflow:Assets written to: iris_binary_classification_model_tf/assets


In [8]:
!python -m tf2onnx.convert --saved-model iris_binary_classification_model_tf --output iris_binary_classification_model.onnx

/home/gatum/.conda/envs/marabou_3/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-11-21 18:34:17,396 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2024-11-21 18:34:17,660 - INFO - Signatures found in model: [serving_default].
2024-11-21 18:34:17,661 - WARNING - '--signature_def' not specified, using first signature: serving_default
2024-11-21 18:34:17,661 - INFO - Output names: ['dense_2']
2024-11-21 18:34:17,760 - INFO - Using tensorflow=2.12.0, onnx=1.17.0, tf2onnx=1.16.1/15c810
2024-11-21 18:34:17,760 - INFO - Using opset <onnx, 15>
2024-11-21 18:34:17,766 - INFO - Computed 0 values for constant folding
2024-11-21 18:34:17,779 - INFO - Optimizing ONNX model
2024-11-21 18:34:17,820 - INFO - After optimization: Identity -2 (2->0)
2024-11-21 18:34:17,821 - INFO - 
2024-11-21